<a href="https://colab.research.google.com/github/luisfelipe100/Tesina/blob/main/V3_YOLOV5_custom_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17488, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17488 (delta 0), reused 1 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17488/17488), 16.59 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (11991/11991), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 232.3 MB/s eta 0:00:01

In [ ]:
!unzip /content/datasets.zip -d /content/

Archive:  /content/datasets.zip
   creating: /content/datasets/
   creating: /content/datasets/monedas/
   creating: /content/datasets/monedas/images/
   creating: /content/datasets/monedas/images/train/
  inflating: /content/datasets/monedas/images/train/0215e6ad-WIN_20250528_20_06_35_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/0728c6ee-WIN_20250528_20_08_08_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/083fa965-WIN_20250528_20_08_29_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/14ba3f69-WIN_20250528_20_08_23_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/166b5c1f-WIN_20250528_20_07_12_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/244b9f4d-WIN_20250528_20_06_46_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/2690f0eb-WIN_20250528_20_08_52_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/35d89297-WIN_20250528_20_07_47_Pro.jpg  
  inflating: /content/datasets/monedas/images/train/

#YOLO Dataset Augmentation
##with Albumentation

In [ ]:
# 🛠 STEP 1: Install dependencies
!pip install albumentations opencv-python tqdm

# 📁 STEP 2: Define source/destination folders
import os

SRC_IMAGES = '/content/datasets/monedas/images/train'     # your original training images
SRC_LABELS = '/content/datasets/monedas/labels/train'     # your original YOLO labels
DST_IMAGES = '/content/datasets/monedas/images_aug/train' # output path for augmented images
DST_LABELS = '/content/datasets/monedas/labels_aug/train' # output path for augmented labels

os.makedirs(DST_IMAGES, exist_ok=True)
os.makedirs(DST_LABELS, exist_ok=True)

# 🧠 STEP 3: Set up augmentations
import albumentations as A

transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    #A.Rotate(limit=180, p=0.5),
    #A.Blur(blur_limit=3, p=0.2),
    #A.HueSaturationValue(p=0.3),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# 🔁 STEP 4: Apply augmentations to each image
import cv2
from tqdm import tqdm

for img_file in tqdm(os.listdir(SRC_IMAGES)):
    if not img_file.endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(SRC_IMAGES, img_file)
    label_path = os.path.join(SRC_LABELS, img_file.replace('.jpg', '.txt').replace('.png', '.txt'))

    # Load image and label
    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
        bboxes = []
        class_labels = []
        for line in lines:
            cls, x, y, bw, bh = map(float, line.strip().split())
            bboxes.append([x, y, bw, bh])
            class_labels.append(int(cls))
    except:
        continue  # skip images without valid labels

    for i in range(2000):  # Generate 2000 augmented images
        try:
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            aug_img = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_labels = augmented['class_labels']
        except Exception as e:
            continue  # skip problematic augmentations

        new_img_name = img_file.replace('.jpg', f'_aug{i}.jpg')
        new_label_name = new_img_name.replace('.jpg', '.txt')

        cv2.imwrite(os.path.join(DST_IMAGES, new_img_name), aug_img)
        with open(os.path.join(DST_LABELS, new_label_name), 'w') as f:
            for label, box in zip(aug_labels, aug_bboxes):
                f.write(f"{label} {' '.join(f'{v:.6f}' for v in box)}\n")


100%|██████████| 28/28 [02:27<00:00,  5.28s/it]


#Visualize YOLO Augmented Images with Boundin Boxes

In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt

# Update with your own paths
image_dir = '/content/datasets/monedas/images_aug/train'
label_dir = '/content/datasets/monedas/labels_aug/train'

# Class names (update these with your actual classes if available)
class_names = ['Moneda_10c', 'Moneda_10p', 'Moneda_1p', 'Moneda_25c', 'Moneda_2p', 'Moneda_50c', 'Pila']

# Helper function to draw boxes
def draw_yolo_box(img, label_path):
    h, w = img.shape[:2]
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x, y, bw, bh = map(float, line.strip().split())
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)
            color = (0, 255, 0)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, class_names[int(class_id)], (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return img

# List and randomly select 10 image files
all_images = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))]
sampled_images = random.sample(all_images, min(50, len(all_images)))

# Display selected images
for img_file in sampled_images:
    image_path = os.path.join(image_dir, img_file)
    label_path = os.path.join(label_dir, img_file.replace('.jpg', '.txt').replace('.png', '.txt'))

    if os.path.exists(label_path):
        img = cv2.imread(image_path)
        img = draw_yolo_box(img, label_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(6, 6))
        plt.imshow(img_rgb)
        plt.title(f"{img_file}")
        plt.axis('off')
        plt.show()
    else:
        print(f"No label found for {img_file}")


#Training the model
##using Transfer Learning with YOLOv5s

In [ ]:
# wandb used to visualize results. You need an account.
# W&B (Weights & Biases) prompts because YOLOv5 logs to it by default. To disable:
import os
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
!python train.py \
  --img 640 \
  --batch 64 \
  --epochs 500 \
  --data /content/datasets/monedas.yaml \
  --weights yolov5s.pt \
  --name monedas_yolov5s \
  --device 0 \
  --workers 8 \
  --cache

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-06 06:27:03.986838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749191224.008476    3124 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749191224.014963    3124 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
!zip -r monedas_yolov5s.zip /content/yolov5/runs/train/monedas_yolov5s


  adding: content/yolov5/runs/train/monedas_yolov5s/ (stored 0%)
  adding: content/yolov5/runs/train/monedas_yolov5s/R_curve.png (deflated 16%)
  adding: content/yolov5/runs/train/monedas_yolov5s/F1_curve.png (deflated 10%)
  adding: content/yolov5/runs/train/monedas_yolov5s/weights/ (stored 0%)
  adding: content/yolov5/runs/train/monedas_yolov5s/weights/best.pt (deflated 9%)
  adding: content/yolov5/runs/train/monedas_yolov5s/weights/last.pt (deflated 9%)
  adding: content/yolov5/runs/train/monedas_yolov5s/results.png (deflated 6%)
  adding: content/yolov5/runs/train/monedas_yolov5s/val_batch0_pred.jpg (deflated 20%)
  adding: content/yolov5/runs/train/monedas_yolov5s/confusion_matrix.png (deflated 27%)
  adding: content/yolov5/runs/train/monedas_yolov5s/results.csv (deflated 85%)
  adding: content/yolov5/runs/train/monedas_yolov5s/labels.jpg (deflated 47%)
  adding: content/yolov5/runs/train/monedas_yolov5s/train_batch0.jpg (deflated 9%)
  adding: content/yolov5/runs/train/monedas_yo